In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import os
import seaborn as sns
import cufflinks as cf
import matplotlib.pyplot as plt
plt.show()
#%matplotlib inline

datasetpath = 'tf_mini.csv'
directory = os.fsencode('playlists')
global descriptions
descriptions = pd.DataFrame()
global popular_clusters
popular_clusters = {}

cf.go_offline()
cf.set_config_file(world_readable=True,theme='pearl')


In [2]:


class k_means_structure:
    def __init__(self, setpath, countrypath, countryname):
        """[CONSTRUCTOR of KMA ]

        Args:
            setpath ([string]): [dataset relative path]
            countrypath ([string]): [relative path of country data json]
            countryname ([string]): [name of country less the jargon of filename]
        """
        self.debug = False
        self.sampleframe = pd.DataFrame(pd.read_csv(setpath), columns=[
                                        'energy', 'valence', 'tempo', 'duration', 'danceability', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness'])  # removed 'us_popularity_estimate', and 'release year'
        self.countryname = countryname
        self.countryframe = pd.DataFrame(pd.read_json(countrypath), columns=[
                                         'energy', 'valence', 'tempo', 'lenght', 'danceability', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness'])
        self.sampleframe['set'] = 2  # SET 2 -> BIG DATASET
        self.countryframe['set'] = 20  # SET 20-> PER COUNTRY TOP 50
        self.countryframe.rename(columns={'lenght': 'duration'}, inplace=True)
        self.countryframe['duration'] = self.countryframe['duration'].div(
            1000).round(2)

        self.fullframe = pd.concat([self.countryframe, self.sampleframe])

        self.kmeans = KMeans(n_clusters=2).fit(self.fullframe)
        self.fullframe['cluster'] = self.kmeans.labels_
        centroids = pd.DataFrame(self.kmeans.cluster_centers_)
        print(centroids)
        #PLOTTING
        self.fullframe.sample(n=50).iplot(kind='scatter3d', x='energy', y='valence', z='tempo', width=0.5,margin=(0,0,0,0),opacity=1,size=15)
        #sns.pairplot(self.fullframe.sample(n=50), hue="cluster")
        
        
        if self.debug is True:
            # DEBUGGING PRINTS
            print(f"LABELS: {self.kmeans.labels_}")
            print(self.fullframe)
            print('COUNT PER CLUSTER', self.fullframe.groupby('cluster').count())

    def return_desc(self):
        """[returns local copy of global variable country descriptions]

        Returns:
            [dict]: [statistical descriptions of every KMA per country]
        """
        self.desc = self.fullframe.groupby('cluster').describe()
        self.desc['country'] = self.countryname[11:-17]
        global descriptions
        descriptions = pd.concat([descriptions, self.desc])
        print('### DESCRIPTION PER CLUSTER ###')
        return descriptions

    def decide_popular_cluster(self):
        """[FIND THE MORE POPULAR CLUSTER PER COUNTRY]

        Returns:
            [dict]: [{country:popular_cluster}]
        """
        global popular_clusters

        cluster0 = 0
        cluster1 = 0
        for index, row in self.fullframe.iterrows():
            if row['cluster'] == 0 and row['set'] == 20:
                cluster0 += 1
            if row['cluster'] == 1 and row['set'] == 20:
                cluster1 += 1
            if cluster0 > cluster1:
                # CLUSTER 0 ES EL POPULAR
                popular_clusters[self.countryname[11:-17]] = 0
            else:
                # CLUSTER 1 ES EL POPULAR
                popular_clusters[self.countryname[11:-17]] = 1

        print(popular_clusters)

        return popular_clusters


def main(pops):

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".json"):
            path = 'playlists/'+filename
            print(path)
            KMA = k_means_structure(datasetpath, path, filename)
            KMA.return_desc()
            KMA.decide_popular_cluster()
            #df = pd.read_excel('countries_massive.xlsx')
    # COMMENTED TO NOT STDOUT to EXCEL XML
    descriptions.to_excel('countries_massive.xlsx')
    # popular cluster -> dataframe -> excel
    pop_cluster = pd.DataFrame.from_dict(list(pops.items()))
    pop_cluster.to_excel('pop_clusters_per_country_massive.xlsx')


In [3]:

main(popular_clusters)


playlists/Top Songs - Argentina Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.642439  0.505927  122.191331  194.208632  0.624800  5.272303 -7.396693   
1  0.638366  0.431212  119.678634  310.681287  0.571145  5.257367 -7.896218   

         7         8         9         10        11  
0  0.125173  0.251948  0.063259  0.198924  2.021018  
1  0.116115  0.245198  0.093425  0.226009  2.007367  


### DESCRIPTION PER CLUSTER ###
{' Argentina': 0}
playlists/Top Songs - Australia Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.638219  0.431103  119.676911  310.710521  0.571097  5.256183 -7.898446   
1  0.642456  0.505831  122.175599  194.225107  0.624703  5.272131 -7.397462   

         7         8         9         10        11  
0  0.116136  0.245304  0.093452  0.226026  2.008845  
1  0.125142  0.251876  0.063253  0.198951  2.020548  


### DESCRIPTION PER CLUSTER ###
{' Argentina': 0, ' Australia': 1}
playlists/Top Songs - Austria Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.642455  0.505860  122.181896  194.203885  0.624762  5.273614 -7.397991   
1  0.638316  0.431125  119.668830  310.717799  0.571078  5.256574 -7.897754   

         7         8         9         10        11  
0  0.125169  0.251930  0.063263  0.198954  2.022414  
1  0.116135  0.245257  0.093475  0.226031  2.002949  


### DESCRIPTION PER CLUSTER ###
{' Argentina': 0, ' Australia': 1, ' Austria': 0}
playlists/Top Songs - Belgium Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.642466  0.505912  122.193548  194.180493  0.624745  5.272647 -7.397707   
1  0.638260  0.431028  119.656648  310.576448  0.571082  5.257374 -7.896440   

         7         8         9         10        11  
0  0.125155  0.251922  0.063252  0.198952  2.020096  
1  0.116035  0.245224  0.093402  0.225882  2.010295  


### DESCRIPTION PER CLUSTER ###
{' Argentina': 0, ' Australia': 1, ' Austria': 0, ' Belgium': 0}
playlists/Top Songs - Brazil Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.642561  0.505948  122.210168  194.381426  0.624618  5.274051 -7.397059   
1  0.638189  0.430584  119.604904  311.295401  0.570894  5.255833 -7.899927   

         7         8         9         10        11  
0  0.125080  0.252021  0.063280  0.199113  2.022810  
1  0.116195  0.245292  0.093689  0.226093  2.001489  


...

### DESCRIPTION PER CLUSTER ###
{' Argentina': 0, ' Australia': 1, ' Austria': 0, ' Belgium': 0, ' Brazil': 0, ' Canada': 1, ' Chile': 1, ' Colombia': 0, ' Czech Republic': 0, ' Denmark': 0, ' Dominican Republic': 0, ' Ecuador': 1, ' Egypt': 1, ' Finland': 0, ' Germany': 1, ' Global': 1, ' Guatemala': 1, ' Hong Kong': 0, ' Hungary': 1, ' India': 1, ' Indonesia': 1, ' Ireland': 0, ' Israel': 0, ' Italy': 0, ' Japan': 0, ' Malaysia': 1, ' Mexico': 1, ' Morocco': 1, ' Netherlands': 1, ' New Zealand': 0, ' Norway': 0, ' Panama': 0, ' Peru': 1, ' Philippines': 1, ' Poland': 1, ' Romania': 0, ' Russia': 1, ' Saudi Arabia': 1, ' Singapore': 0, ' Slovakia': 1, ' South Africa': 1, ' South Korea': 0, ' Spain': 1, ' Sweden': 0, ' Switzerland': 0, ' Taiwan': 0, ' Thailand': 1, ' Turkey': 1, ' UAE': 1}
playlists/Top Songs - Ukraine Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.642437  0.505815  122.194232  194.166009  0.624785  5.272461 -7.3982

### DESCRIPTION PER CLUSTER ###
{' Argentina': 0, ' Australia': 1, ' Austria': 0, ' Belgium': 0, ' Brazil': 0, ' Canada': 1, ' Chile': 1, ' Colombia': 0, ' Czech Republic': 0, ' Denmark': 0, ' Dominican Republic': 0, ' Ecuador': 1, ' Egypt': 1, ' Finland': 0, ' Germany': 1, ' Global': 1, ' Guatemala': 1, ' Hong Kong': 0, ' Hungary': 1, ' India': 1, ' Indonesia': 1, ' Ireland': 0, ' Israel': 0, ' Italy': 0, ' Japan': 0, ' Malaysia': 1, ' Mexico': 1, ' Morocco': 1, ' Netherlands': 1, ' New Zealand': 0, ' Norway': 0, ' Panama': 0, ' Peru': 1, ' Philippines': 1, ' Poland': 1, ' Romania': 0, ' Russia': 1, ' Saudi Arabia': 1, ' Singapore': 0, ' Slovakia': 1, ' South Africa': 1, ' South Korea': 0, ' Spain': 1, ' Sweden': 0, ' Switzerland': 0, ' Taiwan': 0, ' Thailand': 1, ' Turkey': 1, ' UAE': 1, ' Ukraine': 0}
playlists/Top Songs - United Kingdom Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.642494  0.505766  122.186774  194.324316  0.62

### DESCRIPTION PER CLUSTER ###
{' Argentina': 0, ' Australia': 1, ' Austria': 0, ' Belgium': 0, ' Brazil': 0, ' Canada': 1, ' Chile': 1, ' Colombia': 0, ' Czech Republic': 0, ' Denmark': 0, ' Dominican Republic': 0, ' Ecuador': 1, ' Egypt': 1, ' Finland': 0, ' Germany': 1, ' Global': 1, ' Guatemala': 1, ' Hong Kong': 0, ' Hungary': 1, ' India': 1, ' Indonesia': 1, ' Ireland': 0, ' Israel': 0, ' Italy': 0, ' Japan': 0, ' Malaysia': 1, ' Mexico': 1, ' Morocco': 1, ' Netherlands': 1, ' New Zealand': 0, ' Norway': 0, ' Panama': 0, ' Peru': 1, ' Philippines': 1, ' Poland': 1, ' Romania': 0, ' Russia': 1, ' Saudi Arabia': 1, ' Singapore': 0, ' Slovakia': 1, ' South Africa': 1, ' South Korea': 0, ' Spain': 1, ' Sweden': 0, ' Switzerland': 0, ' Taiwan': 0, ' Thailand': 1, ' Turkey': 1, ' UAE': 1, ' Ukraine': 0, ' United Kingdom': 0}
playlists/Top Songs - USA Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.642387  0.505767  122.178843  194.2

### DESCRIPTION PER CLUSTER ###
{' Argentina': 0, ' Australia': 1, ' Austria': 0, ' Belgium': 0, ' Brazil': 0, ' Canada': 1, ' Chile': 1, ' Colombia': 0, ' Czech Republic': 0, ' Denmark': 0, ' Dominican Republic': 0, ' Ecuador': 1, ' Egypt': 1, ' Finland': 0, ' Germany': 1, ' Global': 1, ' Guatemala': 1, ' Hong Kong': 0, ' Hungary': 1, ' India': 1, ' Indonesia': 1, ' Ireland': 0, ' Israel': 0, ' Italy': 0, ' Japan': 0, ' Malaysia': 1, ' Mexico': 1, ' Morocco': 1, ' Netherlands': 1, ' New Zealand': 0, ' Norway': 0, ' Panama': 0, ' Peru': 1, ' Philippines': 1, ' Poland': 1, ' Romania': 0, ' Russia': 1, ' Saudi Arabia': 1, ' Singapore': 0, ' Slovakia': 1, ' South Africa': 1, ' South Korea': 0, ' Spain': 1, ' Sweden': 0, ' Switzerland': 0, ' Taiwan': 0, ' Thailand': 1, ' Turkey': 1, ' UAE': 1, ' Ukraine': 0, ' United Kingdom': 0, ' USA': 0}
playlists/Top Songs - Vietnam Oct-26-2021.json
         0         1           2           3         4         5         6   \
0  0.638150  0.431106  11

### DESCRIPTION PER CLUSTER ###
{' Argentina': 0, ' Australia': 1, ' Austria': 0, ' Belgium': 0, ' Brazil': 0, ' Canada': 1, ' Chile': 1, ' Colombia': 0, ' Czech Republic': 0, ' Denmark': 0, ' Dominican Republic': 0, ' Ecuador': 1, ' Egypt': 1, ' Finland': 0, ' Germany': 1, ' Global': 1, ' Guatemala': 1, ' Hong Kong': 0, ' Hungary': 1, ' India': 1, ' Indonesia': 1, ' Ireland': 0, ' Israel': 0, ' Italy': 0, ' Japan': 0, ' Malaysia': 1, ' Mexico': 1, ' Morocco': 1, ' Netherlands': 1, ' New Zealand': 0, ' Norway': 0, ' Panama': 0, ' Peru': 1, ' Philippines': 1, ' Poland': 1, ' Romania': 0, ' Russia': 1, ' Saudi Arabia': 1, ' Singapore': 0, ' Slovakia': 1, ' South Africa': 1, ' South Korea': 0, ' Spain': 1, ' Sweden': 0, ' Switzerland': 0, ' Taiwan': 0, ' Thailand': 1, ' Turkey': 1, ' UAE': 1, ' Ukraine': 0, ' United Kingdom': 0, ' USA': 0, ' Vietnam': 1}
